### Obsługa wyjątków

Tworzymy sobie pakiet z plikiem do obsługi wyjątków. Np. error.PostNotFoundException, który rozszerza klasę Exception.
Następnie nadpisujemy metody klasy Exception ("alt + insert" w IntelliJ).

In [ ]:
public class PostNotFoundException extends Exception {
    public PostNotFoundException() {
        super();
    }

    public PostNotFoundException(String message) {
        super(message);
    }

    public PostNotFoundException(String message, Throwable cause) {
        super(message, cause);
    }

    public PostNotFoundException(Throwable cause) {
        super(cause);
    }

    protected PostNotFoundException(String message, Throwable cause, boolean enableSuppression, boolean writableStackTrace) {
        super(message, cause, enableSuppression, writableStackTrace);
    }
}

W serwisie modyfikujemy metodę aby mogła obsłużyć błąd np. w przypadku gdy nie ma takiego wpisu w bazie danych. Zwróć uwagę na dodanie "throw" do obsługi wyjątku w serwisie jak i w metodzie Controllera. Przykład poniżej.

In [ ]:
public Post getPostById(Long id) throws PostNotFoundException {
    Optional<Post> post = new postRepository.findById(id);
    
    if (!post.isPresent()) {
        throw new PostNotFoundException("Post is not found");
    }
    
    return post.get();
}

################## In Controller ################
@GetMapping("/posts/{id}")
public Post getPostById(@PathVariable("id") Long id) throws PostNotFoundException  {
    return postService.getPostById(id)
}

Następnie tworzymy model ("Entity") reprezentujący obiekt JSON, który chcemy zwrócić po obsłudze błędu.

In [ ]:
@Getter
@Setter
@NoArgsConstructor
public class ErrorMessage {

    private HttpStatus status;
    private String message;

    public ErrorMessage(HttpStatus status, String message) {
        this.status = status;
        this.message = message;
    }
}

Wracamy do pakietu error, gdzie tworzymy plik RestResponseEntityExceptionHandler, który zwraca odpowiedź w postaci obiektu JSON.

In [ ]:
@ControllerAdvice
@ResponseStatus
public class RestResponseEntityExceptionHandler extends ResponseEntityExceptionHandler {

    @ExceptionHandler(PostNotFoundException.class) ## here we add the class which handles exception
    public ResponseEntity<ErrorMessage> postNotFoundException(PostNotFoundException exception, WebRequest request) {
        ErrorMessage message = new ErrorMessage(HttpStatus.NOT_FOUND, exception.getMessage());

        return ResponseEntity.status(HttpStatus.NOT_FOUND).body(message);
    }